In [44]:
import pandas as pd
from services.db_service import DbService

In [45]:
personal_pool_collection = DbService().personal_pool_collection()
global_pool = DbService().pool_collection()

In [ ]:
df = pd.read_csv("../raw-data/moxfield_wishlist.csv")
df.head()

In [ ]:
df = df[["Count", "Name"]]
df = df.rename(columns={"Count": "count", "Name": "name"})

df.head()

In [ ]:
selected_columns = df[["count", "name"]]
cards = selected_columns.to_dict(orient="records")
cards

In [49]:
import_type = "sum"  # sum or replace


def add_or_update_card(card):
    card_from_pool = global_pool.find_one({"scryfall_name": card["name"]})
    name = card_from_pool["name"] if card_from_pool else card["name"]

    existing_card = personal_pool_collection.find_one({"scryfall_name": card["name"]})
    if not existing_card:
        personal_pool_collection.insert_one(
            {"name": name, "scryfall_name": card["name"], "count": card["count"]})
        return

    if import_type == "sum":
        card["count"] += existing_card["count"]

    personal_pool_collection.update_one({"_id": existing_card["_id"]}, {
                          "$set": {"count": card["count"]}})

In [50]:
for card in cards:
    add_or_update_card(card)